In [36]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.decomposition import NMF

<h1> 1. Get Data </h1>

In [51]:
path  = 'C:/Users/thanh/Documents/CARS/Data/ml-100k/'
df = pd.read_csv(path+ 'u.data', sep = '\t', names=['userID','itemID','rating','timestamp'])
df2= pd.read_csv(path+ 'u.genre', sep = '|',names=['Genres','ID'])
df3=pd.read_csv(path+ 'u.item', sep = '|',encoding = "ISO-8859-1",names=['MovieID','Title','realeasDate','videoReleaseDate','URL','unknown','Action','Adventure', 'Animation',  'Children' , 'Comedy',  'Crime',  'Documentary',  'Drama',  'Fantasy',  'Film-Noir',  'Horror',  'Musical',  'Mystery',  'Romance',  'Sci-Fi',  'Thriller',  'War',  'Western' ])
df = df[:800]
df3 = df3.drop(columns=['Title','realeasDate','videoReleaseDate','URL'])
data = pd.merge(df,df3, left_on='itemID',right_on='MovieID',how='inner').drop(columns='MovieID') #Input for side-info
#number of attributes for feaure "GENRE"
F_genre = len(data.columns)-4
df3 = df3.set_index('MovieID')
zzx = data.groupby(['itemID']).mean().drop(columns=['userID','rating','timestamp']).T
# zzx.columns = zzx.iloc[0]
# zzx = zzx[1:]
rating=data.pivot(index='userID',columns='itemID',values='rating').fillna(0)
rate = rating.reset_index().drop(columns=['userID'])

In [53]:
import numpy as np
import pandas as pd 
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

class MatrixFactorizationCF:
    def __init__(self, n_factors, lambda_reg, n_iterations, alpha):
        self.n_factors = n_factors
        self.lambda_reg = lambda_reg
        self.n_iterations = n_iterations
        self.alpha = alpha
    
    def fit(self, X, item_features):
        model = NMF(n_components=self.n_factors)
        self.n_users, self.n_items = X.shape
        self.R = coo_matrix(X)
        self.item_features = item_features
        self.U = model.fit_transform(self.R)
        self.V = model.components_
        self.item_feature_weights = np.random.normal(size=(self.n_items, self.item_features.shape[1]))
        self.item_feature_bias = np.random.normal(size=self.n_items)
        
        for i in tqdm(range(self.n_iterations)):
            self.update_user_factors()
            self.update_item_factors()
            self.update_item_feature_weights()
            self.update_item_feature_bias()
            
    def predict(self, X):
        return self.U.dot(self.V.T) + self.item_features.dot(self.item_feature_weights.T) + self.item_feature_bias
        
    def update_user_factors(self):
        VV = self.V.T.dot(self.V)
        for i in range(self.n_users):
            V_i = self.V[self.R.row[self.R.row == i], :]
            A = V_i.T.dot(V_i) + self.lambda_reg * np.eye(self.n_factors)
            b = V_i.T.dot(self.R.data[self.R.row == i]) + self.alpha * self.U[i, :]
            self.U[i, :] = np.linalg.solve(A, b)
    
    def update_item_factors(self):
        UU = self.U.T.dot(self.U)
        for j in range(self.n_items):
            U_j = self.U[self.R.col[self.R.col == j], :]
            A = U_j.T.dot(U_j) + self.lambda_reg * np.eye(self.n_factors)
            b = U_j.T.dot(self.R.data[self.R.col == j]) + self.alpha * (self.V[j, :] + self.item_features[j, :].dot(self.item_feature_weights))
            self.V[j, :] = np.linalg.solve(A, b)
    
    def update_item_feature_weights(self):
        for j in range(self.n_items):
            sim = cosine_similarity(self.item_features[j, :].reshape(1, -1), self.item_features)
            w = sim.dot(self.V) / sim.sum()
            A = w.T.dot(w) + self.lambda_reg * np.eye(self.n_factors)
            b = w.T.dot(self.R.data[self.R.col == j] - self.V[j, :].dot(self.U[self.R.row[self.R.col == j], :].T))
            self.item_feature_weights[j, :] = np.linalg.solve(A, b)
    
    def update_item_feature_bias(self):
        for j in range(self.n_items):
            self.item_feature_bias[j] = (self.R.data[self.R.col == j] - self.U[self.R.row[self.R.col == j], :].dot(self.V[j, :].T) - self.item_features[j, :].dot(self.item_feature_weights.T)).mean()


In [55]:
md = MatrixFactorizationCF(n_factors=10,lambda_reg=0.4,alpha=0.012,n_iterations=250)



In [56]:
# Import PyTorch Packages
import torch
from torch import nn
import torch.nn.functional as F


class MF(nn.Module):
    # Iteration counter
    itr = 0

    def __init__(self, n_user, n_item, n_occu, k=10, c_vector=1.0, c_bias=1.0, writer=None):
        """
        :param n_user: User column
        :param n_item: Item column
        :param n_occu: Occupation column
        :param k: Dimensions constant
        :param c_vector: Regularization constant
        :param c_bias: Regularization constant for the biases
        :param writer: Log results via TensorBoard
        """
        super(MF, self).__init__()

        # This will hold the logging
        self.writer = writer

        # These are the hyper-parameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.n_occu = n_occu
        self.c_bias = c_bias
        self.c_vector = c_vector

        # The embedding matrices for user and item are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)

        # We've added new term here: Embedding matrices for occupation vectors
        self.occu = nn.Embedding(n_occu, k)

        # Embedding matrices for the user's biases and the item's biases
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)

        # Initialize the bias tensors
        self.bias = nn.Parameter(torch.ones(1))

    def __call__(self, train_x):
        """This is the most important function in this script"""
        # These are the user indices, and correspond to "u" variable
        user_id = train_x[:, 0]
        # These are the item indices, correspond to the "i" variable
        item_id = train_x[:, 1]

        # Initialize a vector user = p_u using the user indices
        vector_user = self.user(user_id)
        # Initialize a vector item = q_i using the item indices
        vector_item = self.item(item_id)

        # The user-item interaction: p_u * q_i is a dot product between the user vector and the item vector
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)

        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # These are the occupation indices, and correspond to "o" variable
        occu_id = train_x[:, 3]
        # Initialize a vector occupation = r_o using the occupation indices
        vector_occu = self.occu(occu_id)
        # The user-occupation interaction: p_u * r_o is a dot product between the user vector and the occupation vector
        uo_interaction = torch.sum(vector_user * vector_occu, dim=1)

        # Add the bias, the user-item interaction, and the user-occupation interaction to obtain the final prediction
        prediction = ui_interaction + uo_interaction + biases
        return prediction

    def loss(self, prediction, target):
        """
        Function to calculate the loss metric
        """
        # Calculate the Mean Squared Error between target and prediction
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())

        # Compute L2 regularization over the biases for user and the biases for item matrices
        prior_bias_user = l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias

        # Compute L2 regularization over user (P) and item (Q) matrices
        prior_user = l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector

        # Compute L2 regularization over occupation (R) matrices
        prior_occu = l2_regularize(self.occu.weight) * self.c_vector

        # Add up the MSE loss + user & item regularization + user & item biases regularization + occupation
        # regularization
        total = loss_mse + prior_user + prior_item + prior_bias_item + prior_bias_user + prior_occu

        # This logs all local variables to tensorboard
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total


def l2_regularize(array):
    """
    Function to do L2 regularization
    """
    loss = torch.sum(array ** 2.0)
    return loss